# Seminar 2. PyTorch
Hi! Today we are going to study PyTorch. We'll compare numpy and PyTorch commands, rewrite our previous neural network in two ways.

!!! GPU ON !!!

In [ ]:
! pip install numpy scikit-learn matplotlib seaborn mnist torch

In [ ]:
from IPython import display
import numpy as np
import random
import torch

In [ ]:
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

## Numpy vs Pytorch
### Initialization

In [ ]:
a = [1. , 1.4 , 2.5]
print(f"Simple way: {torch.tensor(a)}")
print(f"Zeros:\n {torch.zeros((2,3))}")
print(f"Range: {torch.arange(0, 10)}")
print(f"Complicated range: {torch.arange(4, 12, 2)}")
print(f"Space: {torch.linspace(1, 4, 6)}")
print(f"Identity matrix:\n {torch.eye(4)}")

### Random

In [ ]:
print(f"From 0 to 1: {torch.rand(1)}")
print(f"Vector from 0 to 1: {torch.rand(5)}")
print(f"Vector from 0 to 10: {torch.randint(10, size=(5,))}")

### Matrix Operation

In [ ]:
a = torch.arange(10).type(torch.FloatTensor)
b = torch.linspace(-10, 10, 10)
print(f"a: {a}\nshape: {a.size()}")
print(f"b: {a}\nshape: {b.size()}")
print(f"a + b: {a + b},\n a * b: {a * b}")
print(f"Dot product: {a.dot(b)}")
print(f"Mean: {a.mean()}, STD: {a.std()}")
print(f"Sum: {a.sum()}, Min: {a.min()}, Max: {a.max()}")
print(f"Reshape:\n{a.reshape(-1, 1)}\nshape: {a.reshape(-1, 1).size()}")
c = a.reshape(-1, 1).repeat(1, 5)
print(f"Repeat:\n{c}\nshape: {c.size()}")
print(f"Transpose:\n{c.T}\nshape: {c.T.size()}")
print(f"Unique items: {torch.unique(c)}")

### Indexing

In [ ]:
a = torch.arange(100).reshape(10, 10)
print(f"Array:\n{a}\nshape: {a.size()}")
print(f"Get first column: {a[:, 0]}")
print(f"Get last row: {a[-1, :]}")
print(f"Add new awis:\n{a[:, np.newaxis]}\nshape: {a[:, np.newaxis].size()}")
print(f"Specific indexing:\n{a[4:6, 7:]}")

### Numpy <-> Pytorch

In [ ]:
a = torch.normal(mean=torch.zeros(2,4))
a.numpy()

In [ ]:
b = np.random.normal(size=(2, 4))
torch.from_numpy(b)

### CUDA

In [ ]:
a = torch.normal(mean=torch.zeros(2,4))
b = torch.normal(mean=torch.zeros(2, 4))
print(f"a:\n{a}\nb:\n{b}")

In [ ]:
a = a.cuda()

In [ ]:
a + b

In [ ]:
(a + b.cuda()).cpu()

### Autograd

In [ ]:
a = torch.randn(2, requires_grad=True)
b = torch.normal(mean=torch.zeros(2))

c = torch.dot(a, b)
print(f'a:\n{a}\nb:\n{b}\n(a,b): {c}')

In [ ]:
c.backward()
print(f'a:\n{a}\nb:\n{b}\n(a,b): {c}')

In [ ]:
print(f"Grad a: {a.grad}")

Add function!

In [ ]:
a = torch.randn(2, requires_grad=True)
b = torch.normal(mean=torch.zeros(2))
c = torch.ones(1, requires_grad=True)

d = torch.sigmoid(torch.dot(a, b) + c)
print(f'a:\n{a}\nb:\n{b}\nSigmoid( (a,b) ): {d}')

In [ ]:
print(f"Grad a: {a.grad}\nGrad c: {c.grad}")

In [ ]:
d.backward()
print(f"Grad a: {a.grad}\nGrad c: {c.grad}")

Okay, what about vectors?

In [ ]:
a = torch.randn(2, requires_grad=True)
b = torch.normal(mean=torch.zeros(2))

c = a * b
c.backward()

## Neural Network. Rewind

In [ ]:
from copy import deepcopy

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns

import mnist

sns.set()

In [ ]:
images = mnist.train_images() / 255
labels = mnist.train_labels()

X_train, X_valid, y_train, y_valid = train_test_split(images, labels)

In [ ]:
def get_batches(dataset, batch_size):
    X = dataset[0].reshape(-1, 28 * 28)
    Y = dataset[1]
    n_samples = X.shape[0]
    
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield torch.FloatTensor(X[batch_idx]), torch.LongTensor(Y[batch_idx])

In [ ]:
class CustomLinear:
    def __init__(self, in_size, out_size):
        """
        Simple linear layer
        """
        self.in_size = in_size
        self.out_size = out_size

        self.w = ...  # torch.randn
        self.b = ...  # torch.randn

    def __call__(self, x):
        return ...  # torch.mm

    def zero_grad(self):
        self.w.grad = None
        self.b.grad = None

In [ ]:
class CustomNeuralNetwork:
    def __init__(self, dims, activation="sigmoid"):
        """
        Simple deep networks, that joins several linear layers. 
        """
        self.dims = dims

        self.linears = ...  # list of linear layers
        if activation == "sigmoid":
            self.activation = torch.sigmoid
        elif activation == "relu":
            self.activation = torch.relu
        else:
            raise NotImplementedError

    def __call__(self, x):
        return ...

    def zero_grad(self):
        for l in self.linears:
            l.zero_grad()

In [ ]:
class CustomCrossEntropy:
    def __call__(self, x, target):
        return ...

In [ ]:
def simple_sgd(model, config):
    with torch.no_grad():
        for l in model.linears:
            l.w -= config['learning_rate'] * l.w.grad
            l.b -= config['learning_rate'] * l.b.grad

In [ ]:
net = CustomNeuralNetwork((28 * 28, 10, 10))
criterion = CustomCrossEntropy()

In [ ]:
optimizer_config = {
    "learning_rate": 1e-1
}

In [ ]:
def train(model, optimizer_config, n_epoch=20, batch_size=256):
    train_logs = {"Train Loss": [0,], "Steps": [0,]}
    valid_logs = {"Valid Loss": [0,], "Valid Accuracy": [0,], "Steps": [0,]}
    step = 0
    best_valid_loss = np.inf
    best_model = None

    for i in range(n_epoch):
        for x_batch, y_batch in get_batches((X_train, y_train), batch_size):
            model.zero_grad()
            
            predictions = model(x_batch)
            loss = criterion(predictions, y_batch)

            loss.backward()
            
            simple_sgd(model, optimizer_config)      
            
            step += 1
            train_logs["Train Loss"].append(loss.detach().item())
            train_logs["Steps"].append(step)

        sum_loss = 0
        sum_acc = 0
        count_valid_steps = 0
        with torch.no_grad():
            for x_batch, y_batch in get_batches((X_valid, y_valid), batch_size):
                predictions = model(x_batch)
                loss = criterion(predictions, y_batch)
                sum_loss += loss.item()
                sum_acc += accuracy_score(y_batch, np.argmax(predictions.numpy(), axis=1))
                count_valid_steps += 1

        valid_logs["Valid Loss"].append(sum_loss / count_valid_steps)
        valid_logs["Valid Accuracy"].append(sum_acc / count_valid_steps)
        valid_logs["Steps"].append(step)

        if best_valid_loss > sum_loss / count_valid_steps:
            best_valid_loss = sum_loss / count_valid_steps
            best_model = deepcopy(model)

    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    sns.lineplot(x="Steps", y="Train Loss", data=train_logs, ax=ax[0])
    sns.lineplot(x="Steps", y="Valid Loss", data=valid_logs, ax=ax[1])
    sns.lineplot(x="Steps", y="Valid Accuracy", data=valid_logs, ax=ax[2])
    plt.plot()

    return best_model, train_logs, valid_logs

In [ ]:
net, _, _ = train(net, optimizer_config)

## Neural Network. Rewind #2

In [ ]:
import torch.nn as nn


class Linear(nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()

        self.in_size = in_size
        self.out_size = out_size

        self.w = ...  # nn.Parameter
        self.b = ...  # nn.Parameter

    def forward(self, x):
        return ...  # torch.mm


class BatchNorm(nn.Module):
    def __init__(self, in_size, alpha=0.1):
        super().__init__()

        self.in_size = in_size

        self.beta = ...  # nn.Parameter
        self.gamma = ...  # nn.Parameter

        self.epsilon = 1e-5

    def forward(self, x):
        return ...


class Dropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()

        self.p = p

    def forward(self, x):
        return ...


class Block(nn.Module):
    def __init__(self, in_size, out_size, activation="sigmoid", p=0.5):
        super().__init__()

        self.linear = Linear(in_size, out_size)
        self.dropout = Dropout(p)
        self.batch_norm = BatchNorm(out_size)
        
        if activation == "sigmoid":
            self.activation = torch.sigmoid
        elif activation == "relu":
            self.activation = torch.relu
        else:
            raise NotImplementedError

    def forward(self, x):
        x = ...
        x = ...
        x = ...
        return ...


class Net(nn.Module):
    def __init__(self, dims, activation="relu", p=0.5):
        super().__init__()

        self.blocks = nn.ModuleList(
            list(Block(d_0, d_1, activation=activation, p=p) for d_0, d_1 in zip(dims[:-2], dims[1:-1]))
        )
        self.cl = Linear(dims[-2], dims[-1])

    def forward(self, x):
        for m in self.blocks:
            x = m(x)
        return self.cl(x)

In [ ]:
net = Net((28 * 28, 100, 10), p=0.1).cuda()
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)

In [ ]:
def train(model, optimizer, n_epoch=20, batch_size=256, device="cpu"):
    train_logs = {"Train Loss": [0,], "Steps": [0,]}
    valid_logs = {"Valid Loss": [0,], "Valid Accuracy": [0,], "Steps": [0,]}
    step = 0
    best_valid_loss = np.inf
    best_model = Nonein

    for i in range(n_epoch):
        for x_batch, y_batch in get_batches((X_train, y_train), batch_size):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            
            predictions = model(x_batch)
            loss = criterion(predictions, y_batch)

            loss.backward()
            
            optimizer.step()   
            
            step += 1
            train_logs["Train Loss"].append(loss.detach().item())
            train_logs["Steps"].append(step)

        sum_loss = 0
        sum_acc = 0
        count_valid_steps = 0
        with torch.no_grad():
            for x_batch, y_batch in get_batches((X_valid, y_valid), batch_size):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                predictions = model(x_batch)
                loss = criterion(predictions, y_batch)
                sum_loss += loss.item()
                sum_acc += accuracy_score(y_batch.cpu().numpy(), np.argmax(predictions.cpu().numpy(), axis=1))
                count_valid_steps += 1

            valid_logs["Valid Loss"].append(sum_loss / count_valid_steps)
            valid_logs["Valid Accuracy"].append(sum_acc / count_valid_steps)
            valid_logs["Steps"].append(step)

            if best_valid_loss > sum_loss / count_valid_steps:
                best_valid_loss = sum_loss / count_valid_steps
                best_model = deepcopy(net)

    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    sns.lineplot(x="Steps", y="Train Loss", data=train_logs, ax=ax[0])
    sns.lineplot(x="Steps", y="Valid Loss", data=valid_logs, ax=ax[1])
    sns.lineplot(x="Steps", y="Valid Accuracy", data=valid_logs, ax=ax[2])
    plt.plot()

    return best_model, train_logs, valid_logs

In [ ]:
net, _, _ = train(net, optimizer, device="cuda:0")

### Neural Network. Rewind #3. Logging

Logging systems:
- [Tensorboard](https://pytorch.org/docs/stable/tensorboard.html)
- [WandB](https://www.wandb.com/)

In [ ]:
class Block(nn.Module):
    def __init__(self, in_size, out_size, activation="relu", p=0.5):
        super().__init__()

        self.in_size = in_size
        self.out_size = out_size

        if activation == "sigmoid":
            self.activation = nn.Sigmoid
        elif activation == "relu":
            self.activation = nn.ReLU
        else:
            raise NotImplementedError

        self.fc = nn.Sequential(
            ...  # Linear, BatchNorm1d, Dropout, Activation
        )

    def forward(self, x):
        return self.fc(x)


net = nn.Sequential(Block(28 * 28, 100, p=0.2), nn.Linear(100, 10)).cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
criterion = nn.CrossEntropyLoss()

In [ ]:
%load_ext tensorboard

In [ ]:
from datetime import datetime
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter

In [ ]:
def train(model, optimizer, n_epoch=20, batch_size=256, device="cpu"):
    writer = SummaryWriter(Path("logs") / datetime.now().strftime("%Y%m%d-%H%M%S"))
    step = 0
    best_valid_loss = np.inf
    best_model = None

    for i in range(n_epoch):
        for x_batch, y_batch in get_batches((X_train, y_train), batch_size):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            
            predictions = model(x_batch)
            loss = criterion(predictions, y_batch)

            loss.backward()
            
            optimizer.step()   
            
            step += 1
            writer.add_scalar("Train Loss", loss.detach().item(), step)

        sum_loss = 0
        sum_acc = 0
        count_valid_steps = 0
        with torch.no_grad():
            for x_batch, y_batch in get_batches((X_valid, y_valid), batch_size):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                predictions = model(x_batch)
                loss = criterion(predictions, y_batch)
                sum_loss += loss.item()
                sum_acc += accuracy_score(y_batch.cpu().numpy(), np.argmax(predictions.cpu().numpy(), axis=1))
                count_valid_steps += 1

            writer.add_scalar("Valid Loss", sum_loss / count_valid_steps, step)
            writer.add_scalar("Valid Accuracy", sum_acc / count_valid_steps, step)

            if best_valid_loss > sum_loss / count_valid_steps:
                best_valid_loss = sum_loss / count_valid_steps
                best_model = deepcopy(net)

    return best_model

In [ ]:
net = train(net, optimizer, device="cuda:0")

In [ ]:
%tensorboard --logdir logs

## Spoiler - train loop with [Catalyst](https://github.com/catalyst-team/catalyst)

- [A comprehensive step-by-step guide to basic and advanced features](https://github.com/catalyst-team/catalyst#step-by-step-guide)
- [Docs](https://catalyst-team.github.io/catalyst/)
- [What is Runner?](https://catalyst-team.github.io/catalyst/api/core.html#runner)

In [ ]:
! pip install catalyst==21.04.2

In [ ]:
import typing as tp
from datetime import datetime
import os
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from catalyst import dl, metrics, utils
from catalyst.contrib.datasets import MNIST
from catalyst.data.transforms import ToTensor

model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10))
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
criterion = nn.CrossEntropyLoss()

loaders = {
    "train": DataLoader(
        MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()), batch_size=32,
    ),
    "valid": DataLoader(
        MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()), batch_size=32,
    ),
}


class CustomRunner(dl.Runner):
    def predict_batch(self, batch: tp.Tuple[torch.Tensor, torch.Tensor]) -> torch.Tensor:
        # model inference step
        return self.model(batch[0].to(self.device))

    def on_loader_start(self, runner):
        super().on_loader_start(runner)
        self.meters = {
            key: metrics.AdditiveValueMetric(compute_on_call=False)
            for key in ["loss", "accuracy01", "accuracy03"]
        }

    def handle_batch(self, batch: tp.Tuple[torch.Tensor, torch.Tensor]):
        # model train/valid step
        # unpack the batch
        x, y = batch
        # run model forward pass
        y_hat = self.model(x)
        # compute the loss
        loss = self.criterion(y_hat, y)
        # compute the metrics
        accuracy01, accuracy03 = metrics.accuracy(y_hat, y, topk=(1, 3))
        # log metrics
        self.batch_metrics.update(
            {"loss": loss, "accuracy01": accuracy01, "accuracy03": accuracy03}
        )
        for key in ["loss", "accuracy01", "accuracy03"]:
            self.meters[key].update(self.batch_metrics[key].item(), self.batch_size)
        # run model backward pass
        if self.is_train_loader:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

    def on_loader_end(self, runner):
        for key in ["loss", "accuracy01", "accuracy03"]:
            self.loader_metrics[key] = self.meters[key].compute()[0]
        super().on_loader_end(runner)


runner = CustomRunner()

# model training
runner.train(
    engine=dl.DeviceEngine(device=utils.get_device()),
    model=model,
    optimizer=optimizer,
    loaders=loaders,
    criterion=criterion,
    logdir=Path("logs") / datetime.now().strftime("%Y%m%d-%H%M%S"),
    num_epochs=5,
    verbose=True,
    load_best_on_end=True,
)

# model inference
for prediction in runner.predict_loader(loader=loaders["valid"]):
    assert prediction.detach().cpu().numpy().shape[-1] == 10

features_batch = next(iter(loaders["valid"]))[0]
# model tracing
utils.trace_model(model=runner.model, batch=features_batch)
# model quantization
utils.quantize_model(model=runner.model)
# model pruning
utils.prune_model(model=runner.model, pruning_fn="l1_unstructured", amount=0.8)
# onnx export
# utils.onnx_export(model=runner.model, batch=features_batch, file="./logs/mnist.onnx", verbose=True)